In [1]:
import argparse
import os
import pickle

import mlflow
from urllib.parse import urlparse
import numpy as np
from hyperopt import STATUS_OK, Trials, SparkTrials, fmin, hp, tpe
from hyperopt.pyll import scope
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

mlflow.set_experiment("RandomForestHyperopt-GreenTaxi")
mlflow.set_tracking_uri("http://ec2-15-222-242-133.ca-central-1.compute.amazonaws.com:5000/")

In [2]:
def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

In [3]:

data_path = "../output"
X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))
X_test, y_test = load_pickle(os.path.join(data_path, "test.pkl"))

In [4]:
def train_and_evaluate(X_train, y_train, X_valid, y_valid, X_test, y_test, params):
    
    with mlflow.start_run(nested=True) as run:

        rf = RandomForestRegressor(**params)
        rf.fit(X_train, y_train)
       
        metrics_val = mean_squared_error(y_valid, rf.predict(X_valid), squared=False)
        metrics_test = mean_squared_error(y_test, rf.predict(X_test), squared=False)
        
        metrics = {"validation_rmse": metrics_val, "test_rmse": metrics_test}
        mlflow.log_params(params)
        mlflow.log_metric( "validation_rmse", metrics_val)
        mlflow.log_metric( "test_rmse", metrics_test)
    
           
        return metrics
    

In [5]:
def objective(X_train, y_train, X_valid, y_valid, X_test, y_test, metric):
    
    def eval(params):
        metrics = train_and_evaluate(
            X_train, y_train, X_valid, y_valid, X_test, y_test, params)
        return {"status": STATUS_OK, "loss": metrics[metric]}
    
    return eval


def log_best(run: mlflow.entities.Run, metric: str) -> None:
    """Log the best parameters from optimization to the parent experiment.

    Args:
        run: current run to log metrics
        metric: name of metric to select best and log
    """

    client = mlflow.tracking.MlflowClient()
    runs = client.search_runs(
        [run.info.experiment_id],
        "tags.mlflow.parentRunId = '{run_id}' ".format(run_id=run.info.run_id))

    best_run = min(runs, key=lambda run: run.data.metrics[metric])

    mlflow.set_tag("best_run", best_run.info.run_id)
    mlflow.log_metric(f"best_{metric}", best_run.data.metrics[metric])
    

In [16]:
METRIC = "test_rmse"
PARALLELISM = 8
search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
        'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
        'random_state': 42
}

trials = SparkTrials(parallelism=PARALLELISM)
train_objective = objective(
        X_train, y_train, X_valid, y_valid, X_test, y_test, METRIC)
rstate = np.random.default_rng(42)  # for reproducible results
with mlflow.start_run() as run:
        fmin(
                fn=train_objective,
                space=search_space,
                algo=tpe.suggest,
                max_evals=50,
                trials=trials,
                rstate=rstate
        )
        ##log_best(run, METRIC)
        search_run_id = run.info.run_id
        experiment_id = run.info.experiment_id


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

  2%|▏         | 1/50 [00:13<10:40, 13.08s/trial, best loss: 7.423891644564951]

  4%|▍         | 2/50 [00:20<07:36,  9.51s/trial, best loss: 6.751804438392731]

  6%|▌         | 3/50 [00:35<09:25, 12.04s/trial, best loss: 6.644020610422899]

  8%|▊         | 4/50 [00:46<08:55, 11.64s/trial, best loss: 6.634872508348875]

...

 94%|█████████▍| 47/50 [06:08<00:21,  7.02s/trial, best loss: 6.549614107984208]

 96%|█████████▌| 48/50 [06:10<00:11,  5.57s/trial, best loss: 6.549614107984208]

 98%|█████████▊| 49/50 [06:12<00:04,  4.53s/trial, best loss: 6.549614107984208]

100%|██████████| 50/50 [06:32<00:00,  7.85s/trial, best loss: 6.548287153830625]


Total Trials: 50: 50 succeeded, 0 failed, 0 cancelled.
